In [2]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [9]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNHybrid
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

In [11]:
args = argparse.Namespace()

# Data
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 32
args.validation_split = 0.05
args.labels_to_categorical = 1
args.opt = 'Ranger'

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.ansatz_class = "NQubit"
args.sparse = 1

In [7]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 8, 8, 1) │ (634, 8, 8, 1) │ (2115, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031, 2)       │ (634, 2)       │ (2115, 2)       │ (2,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.58 │ 112.53 │      0 │  

In [8]:
data.config()

{'input_dims': (8, 8, 1), 'output_dims': (2,), 'mapping': [0, 1]}

In [12]:
model = QCNNHybrid(data.config(), args)

In [13]:
model.compile()
model.build_graph().summary()

Model: "QCNNHybrid-AngleMap-Chen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 8, 1)]         0         
                                                                 
 qconv2d_1 (QConv2D)         (None, 8, 8, 1, 1)        45        
                                                                 
 qconv2d_2 (QConv2D)         (None, 8, 8, 1, 1)        20        
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 2)    

In [14]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
  2/376 [..............................] - ETA: 26:04 - loss: 0.6933 - auc_1: 0.5339 - accuracy: 0.5469

KeyboardInterrupt: 

In [ ]:
model.test(data, callbacks=callbacks)